In [1]:
# Import BeautifulSoup
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

# Import Splinter and set the chromedriver path
from splinter import Browser

### Define useful functions
* visit_scrape_soup - Visit the url and scrape into soup
* scrape_hemisphere - Visit the USGS Astrogeology site to obtain high resolution images for the requested Mars hemisphere.

In [2]:
sleeptime = 0.3
def visit_scrape_soup(b, url):
    '''
    Visit the url and scrape into soup
    '''
    b.visit(url)
    time.sleep(sleeptime)
    # Scrape the browser into soup
    html = b.html
    return bs(html, 'lxml')
# def visit_scrape_soup(b, url)

def scrape_hemisphere(b, num):
    '''
    Mars Hemispheres:
    Visit the USGS Astrogeology site to obtain high resolution images for the requested Mars hemisphere.
    '''
    # Design an XPATH selector to grab the hemisphere images
    xpath = '//div[@class="collapsible results"]/div[@class="item"]/a/img'

    # Find links to hemisphere image thumbnails and click on the requested one
    hresults = b.find_by_xpath(xpath)
    hresults[num].click()
    time.sleep(sleeptime)

    # Scrape the browser into soup
    html = b.html
    soup = bs(html, 'lxml')

    # Save title and url in dict
    imgdict = dict()
    imgdict["title"] = soup.find("h2", class_="title").text.strip("Enhanced").strip()
    imgdict["url"] = soup.find("div", class_="downloads").ul.li.a["href"]

    # Go back to previous page
    b.back()
    return imgdict
# def scrape_hemisphere(b, num)

# Mission to Mars

In [3]:
# Start the browser
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

### NASA Mars News
* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text.

In [4]:
# Visit the URL and scrape
mnewsurl = "https://mars.nasa.gov/news/"
nsoup = visit_scrape_soup(browser, mnewsurl)

# Find the first news title and paragraph
news = nsoup.find("li", class_="slide")
news_title = news.find('div', class_='content_title').text
news_p = news.find('div', class_='rollover_description_inner').text

print(f'{news_title} : \n{news_p}')

Beyond Mars, the Mini MarCO Spacecraft Fall Silent : 
The first CubeSat mission to deep space may have reached their limit, but they could inspire future NASA missions.


### JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars) and find the image url for the current Featured Mars Image.

In [5]:
jplurl = "https://www.jpl.nasa.gov"
# Visit the URL and scrape
img_search_url = f"{jplurl}/spaceimages/?search=&category=Mars"
imgsoup = visit_scrape_soup(browser, img_search_url)

# Find path to wallpaper size image of the current Featured Mars Image
imgitem = imgsoup.find("article", class_="carousel_item")
imgpath = imgitem['style'].split("'")[1]
imgurl = f"{jplurl}{imgpath}"

print(imgurl)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA08003-1920x1200.jpg


### Mars Weather
* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page.

In [6]:
# Visit the URL
wurl = "https://twitter.com/marswxreport?lang=en"
wsoup = visit_scrape_soup(browser, wurl)

# Get list of tweets
tlist = wsoup.find_all("li", class_="js-stream-item")
wtext = None
wkeywords = {'Sol', 'pressure', 'daylight'}

# Loop through and find the most recent weather tweet
for t in tlist:
    if t.div["data-screen-name"] == "MarsWxReport":
        mwtext = t.find(class_="tweet-text").a.previousSibling
        if wkeywords.issubset(set(mwtext.split())):
            wtext = mwtext
            break

print(wtext)

Sol 2308 (2019-02-02), high -13C/8F, low -71C/-95F, pressure at 8.14 hPa, daylight 06:47-18:54


### Mars Facts
* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

In [7]:
# Use Panda's `read_html` to parse the url
furl = "http://space-facts.com/mars/"
ftables = pd.read_html(furl)

# Get the table dataframe and update column names
fdf = ftables[0]
fdf.columns = ['Parameter', 'Value']

# Use to_html to generate HTML tables from dataframe.
fhtml = fdf.to_html(index=False,
                    justify="center",
                    classes="table table-striped table-hover table-dark table-bordered table-sm")

# Strip unwanted newlines to clean up the table.
fhtml = fhtml.replace('\n', '')

print(fhtml)

<table border="1" class="dataframe table table-striped table-hover table-dark table-bordered table-sm">  <thead>    <tr style="text-align: center;">      <th>Parameter</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


### Mars Hemispheres
* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

In [8]:
hurl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
# Visit the URL
browser.visit(hurl)
time.sleep(sleeptime)

# list of dicts to save results
himglist = list()
hnum = 0
while hnum < 4:
    himglist.append(scrape_hemisphere(browser, hnum))
    hnum += 1

print(himglist)

[{'title': 'Cerberus Hemisphere', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [9]:
# Close the browser after scraping
browser.quit()